<a href="https://colab.research.google.com/github/Renatojcalpalhao/Csharp-COMPLETO-2020-Programacao-Orientada-a-Objetos_Projetos/blob/master/Copy_of_Scraper2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#codigo usando o scraper
import scrapy

class PokemonScrapper(scrapy.Spider):
    name = 'pokemon_scrapper'
    domain = "https://pokemondb.net"

    start_urls = ["https://pokemondb.net/pokedex/all"]

    def parse(self, response):
        pokemons = response.css('#pokedex > tbody > tr')

        for pokemon in pokemons:
            link = pokemon.css("td.cell-name > a::attr(href)").extract_first()
            full_link = urljoin(self.domain, link)

            yield response.follow(full_link, self.parse_pokemon)

    def parse_pokemon(self, response):
        # Extraindo as informações principais do Pokémon
        pokemon_id = response.css(
            '.vitals-table > tbody > tr:nth-child(1) > td > strong::text').get(
            )

        pokemon_name = response.css('#main > h1::text').get()
        pokemon_types = response.css(
            '.vitals-table > tbody > tr:nth-child(2) > td > a::text').getall()
        pokemon_height = response.css(
            '.vitals-table > tbody > tr:nth-child(4) > td::text').get().split(
            )[0]
        pokemon_weight = response.css(
            '.vitals-table > tbody > tr:nth-child(5) > td::text').get().split(
            )[0]
        # Extraindo as evoluções
        evolutions = []
        evolution_cards = response.css(
            '#main > div.infocard-list-evo > div.infocard')
        for card in evolution_cards:
            evo_name = card.css(
                'span.infocard-lg-data > a.ent-name::text').get()
            evo_number = card.css('span.infocard-lg-data > small::text').get()
            evo_url = urljoin(self.domain,
                              card.css('a.ent-name::attr(href)').get())
            evolutions.append({
                'number': evo_number,
                'name': evo_name,
                'url': evo_url
            })
        # Extraindo as habilidades
        abilities = []
        ability_links = response.css(
            '.vitals-table a[href*="/ability/"]::attr(href)').getall()
        for link in ability_links:
            full_link = urljoin(self.domain, link)
            abilities.append(full_link)
        # Armazenando os dados do Pokémon
        pokemon_data = {
            'pokemon_id': pokemon_id,
            'pokemon_name': pokemon_name,
            'types': pokemon_types,
            'height_cm': pokemon_height,
            'weight_kg': pokemon_weight,
            'evolutions': evolutions,
            'abilities': abilities
        }
        # Seguindo os links das habilidades para extrair mais informações
        for ability_link in abilities:
            yield response.follow(ability_link,
                                  self.parse_ability,
                                  meta={'pokemon_data': pokemon_data})

    def parse_ability(self, response):
        ability_name = response.css('#main > h1::text').get()
        pokemon_data = response.meta['pokemon_data']
        # Adicionando a habilidade ao Pokémon
        if 'abilities_details' not in pokemon_data:
            pokemon_data['abilities_details'] = []
        pokemon_data['abilities_details'].append(
            {'ability_name': ability_name})
        # Retornando os dados completos do Pokémon
        yield pokemon_data
